In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import subprocess
!pip install shap
!pip install --upgrade shap

try:
    import japanize_matplotlib
    print("japanize_matplotlib is already installed.")
except ImportError:
    !pip install japanize_matplotlib
    print("japanize_matplotlib has been installed.")

try:
    import catboost
    print("catboost is already installed.")
except ImportError:
    !pip install catboost
    print("catboost has been installed.")

try:
    import optuna
    print("optuna is already installed.")
except ImportError:
    !pip install optuna
    print("optuna has been installed.")

try:
    import geolib
    print("geolib is already installed.")
except ImportError:
    !pip install geolib
    print("geolib has been installed.")

# mkdirとechoコマンドの実行
try:
    subprocess.check_call('mkdir -p /etc/OpenCL/vendors && echo "libnvidia-opencl.so.1" > /etc/OpenCL/vendors/nvidia.icd', shell=True)
    print('Directory and file for OpenCL vendors have been created.')
except subprocess.CalledProcessError as e:
    print(f"Error running mkdir and echo command: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.1/540.1 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 38.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for japanize_matplotlib: filename=japanize_matplotlib-1.1.3-py3-none-any.whl size=4120257 sha256=f25df4ee3e8e6eadd1980d9f81243c1bd587379e5935201c20c9fc0d0e8a608d
  Stored in directory: /root/.cache/pip/wheels/61/7a/6b/df1f79be9c59862525070e157e62b08eab8ece27c1b68fbb94
Successfully built japanize_matplotlib
japanize_matplotlib has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 14.1 MB/s eta 0:00:00
catboost has been installed.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.8/362.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 6.3 MB/s eta 0:00:00
optuna has been installed.
geolib has been installed.
Directory and file for OpenCL vendo

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import joblib

import sys
sys.path.append('/content/drive/MyDrive/yuto/project_directory/code')

from utils import (ProcessData,
                   trans_chara,
                   fill_missing_values,
                   fill_missing_with_mode,
                   find_nearest_station_main_table,
                   calculate_distance,
                   convert_kanji_to_fullwidth_2,
                   encode_geohash,
                   to_cat_col,
                   TreeExecution,
                   VotingModel,
                   submisson_datetime,
                   check_dataframe_elements
                   )

TOTAL_PRICE_LOG="取引価格（総額）_log"
PRICE_PER_AREA_LOG="単位面積あたりの取引価格_log"

#TARGET="取引価格（総額）_log"

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/preprocess_main_table.csv",index_col=0)
df=df[['dataset', 'ID', '価格情報区分', '都道府県名', '市区町村名', '地区名', '住所不一致_Flag', '住所',
       '最寄駅：名称', '最寄駅：距離（分）', '徒歩時間_10分単位', 'x_api', 'y_api', 'lon', 'lat',
       '間取り', '面積（㎡）', '建築年', '建物の構造', '用途', '今後の利用目的', '都市計画', '建ぺい率（％）',
       '容積率（％）', '取引時点', '改装', '取引の事情等', '単位面積あたりの取引価格_log', '建築西暦年', '築年数',
       'log_面積（㎡）', '購入までの築年数', '面積（㎡）-最寄駅：距離（分）', '面積（㎡）-築年数',
       '面積（㎡）容積率（％）_combi', '旧耐震フラグ', '最寄駅までの距離', 'geohash_2', 'geohash_3',
       'geohash_4', 'geohash_5', 'geohash_6', 'geohash_7', '県_市_count',
       '建築年_count', '県_市_geohash_2_駅数', '県_市_geohash_3_駅数', '県_市_geohash_4_駅数',
       '県_市_geohash_5_駅数', '県_市_geohash_6_駅数', '県_市_geohash_7_駅数',
       '県_市_geohash_2_駅徒歩時間平均', '県_市_geohash_2_建築年平均', '県_市_geohash_3_駅徒歩時間平均',
       '県_市_geohash_3_建築年平均', '県_市_geohash_4_駅徒歩時間平均', '県_市_geohash_4_建築年平均',
       '県_市_geohash_5_駅徒歩時間平均', '県_市_geohash_5_建築年平均', '県_市_geohash_6_駅徒歩時間平均',
       '県_市_geohash_6_建築年平均', '県_市_geohash_7_駅徒歩時間平均', '県_市_geohash_7_建築年平均']]


In [ ]:
# データフレームを分割
df_train = df[df['dataset'] == 'train'].set_index('ID')
df_test = df[df['dataset'] == 'test'].set_index('ID')

# datasetカラムを削除
df_train = df_train.drop(columns=['dataset'])
df_test = df_test.drop(columns=['dataset'])

df_train,cat_cols=to_cat_col(df_train)

# インデックスを読み込み
train_index = np.load('/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/train_index_l1_cv.npy')
test_index = np.load('/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/checkpoints/test_index_l1_cv.npy')
print(train_index)
print(test_index)
# インデックスを設定して順序を再現
df_train = df_train.loc[train_index]
df_test = df_test.loc[test_index]

print("Train DataFrame:")
print(df_train)
print("\nTest DataFrame:")
print(df_test)

[ 1046933  1079053  1170588 ... 47003811 47010902 47011639]
[ 1000107  1000109  1000112 ... 47037850 47037855 47037856]
Train DataFrame:
             価格情報区分 都道府県名   市区町村名    地区名  住所不一致_Flag              住所 最寄駅：名称  \
ID                                                                            
1046933   不動産取引価格情報   北海道  札幌市白石区    南郷通           0    北海道札幌市白石区南郷通  南郷７丁目   
1079053   不動産取引価格情報   北海道   札幌市西区  発寒１２条           0   北海道札幌市西区発寒１２条   発寒中央   
1170588      成約価格情報   北海道     江別市   大麻北町           0      北海道江別市大麻北町   森林公園   
1001637      成約価格情報   北海道  札幌市中央区   北５条西           0   北海道札幌市中央区北５条西  西２８丁目   
1014790   不動産取引価格情報   北海道  札幌市中央区  宮の森２条           0  北海道札幌市中央区宮の森２条  西２８丁目   
...             ...   ...     ...    ...         ...             ...    ...   
47001427  不動産取引価格情報   沖縄県     那覇市  首里山川町           0     沖縄県那覇市首里山川町     儀保   
47035975     成約価格情報   沖縄県  中頭郡北谷町     美浜           0     沖縄県中頭郡北谷町美浜  てだこ浦西   
47003811  不動産取引価格情報   沖縄県     那覇市    古波蔵           0       沖縄県那覇市古波蔵     

In [ ]:
tree_class=TreeExecution(df_train,y_name=PRICE_PER_AREA_LOG,cat_cols=cat_cols)
#model_lgbm=tree_class.lightgbm_simple()
fitted_models_lgbm,mae_score,oof_df=tree_class.lightgbm_l1_cv()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
oof_df.to_csv("/content/drive/MyDrive/yuto/project_directory/data/cv_predictions/sub_oof_lgbm_l1_cv.csv")
oof_df

,target,pred_lgbm
ID,,
1046933,5.243038,5.329627
1079053,5.124939,5.059953
1170588,4.867762,5.075637
1001637,5.484674,5.486013
1014790,5.260601,5.267016
...,...,...
47001427,5.342423,5.369479
47035975,5.726999,5.687249
47003811,5.528274,5.657619


In [ ]:
#model_cat=VotingModel(fitted_models_cat,cat_cols)
model_lgbm = VotingModel(fitted_models_lgbm,cat_cols)
#model_xg=VotingModel(fitted_models_xg,cat_cols)
#y_pred_cat=pd.Series(model_cat.predict_cat(df_test.drop(f"{TARGET}",axis=1)),index=df_test.index)
y_pred_lgbm=pd.Series(model_lgbm.predict_lgbm(df_test.drop(f"{PRICE_PER_AREA_LOG}",axis=1)),index=df_test.index)
#y_pred_xg=pd.Series(model_xg.predict_xg(df_test_tree.drop("取引価格（総額）_log",axis=1)),index=df_test_tree.index)


y_pred=y_pred_lgbm
# 単位面積あたりの取引価格の対数から10のべき乗を取る
price_per_area = 10**y_pred_lgbm
# その値に面積を掛けて、取引価格（総額）を計算
predicted_price_total = price_per_area * df_test["面積（㎡）"]
# 取引価格（総額）の対数を計算
y_pred= np.log10(predicted_price_total)
#y_pred=y_pred.rename(columns={"面積（㎡）":"取引価格（総額）_log"})
# Seriesの名前を変更
y_pred.name = "取引価格（総額）_log"

#y_pred=y_pred_cat
#y_pred=y_pred_lgbm
# y_predをデータフレームに変換
df_pred = pd.DataFrame(y_pred, columns=["取引価格（総額）_log"])
#print(df_pred)
#print("--------------------------------------------------------------------------------")
# インデックスをリセットし、カラム名を'ID'に変更
#df_pred.reset_index(inplace=True)
#print(df_pred)
#print("--------------------------------------------------------------------------------")
#df_pred.rename(columns={'index': 'ID'}, inplace=True)
#print(df_pred)
#print("--------------------------------------------------------------------------------")

# 結果を表示
print(df_pred)

now_str=submisson_datetime()
df_pred.to_csv("/content/drive/MyDrive/yuto/project_directory/data/intermediate_results/one_model_predictions/sub_test_lgbm_l1_cv.csv")


          取引価格（総額）_log
ID                    
1000107       6.754242
1000109       7.312165
1000112       6.674249
1000114       6.474699
1000117       7.387694
...                ...
47036620      7.519506
47037115      7.450802
47037850      7.241828
47037855      7.523497
47037856      7.563230

[54402 rows x 1 columns]


In [ ]:
df_sub=pd.read_csv("/content/drive/MyDrive/yuto/project_directory/data/final_predictions/test_lgbm_cv20240908_111634.csv")
print(df_sub)

             ID  取引価格（総額）_log
0       1000107      6.754242
1       1000109      7.312165
2       1000112      6.674249
3       1000114      6.474699
4       1000117      7.387694
...         ...           ...
54397  47036620      7.519506
54398  47037115      7.450802
54399  47037850      7.241828
54400  47037855      7.523497
54401  47037856      7.563230

[54402 rows x 2 columns]


In [ ]:
def ML_Record(df, model, cv_score, now_str, model_name):
    # 例として、取得したカラム名のリスト
    columns = df.columns.tolist()

    # モデルがアンサンブルか単一のモデルかをチェック
    if hasattr(model, 'estimators'):
        # 各estimatorのパラメータを取得し、全て同じかチェック
        params_list = [estimator.get_params() for estimator in model.estimators]
        # 最初のモデルのパラメータを基準に比較
        base_params = params_list[0]
        all_same = all(params == base_params for params in params_list)

        if all_same:
            print("All estimators have the same parameters.")
        else:
            print("Not all estimators have the same parameters.")

        # 各estimatorのパラメータを表示
        for i, params in enumerate(params_list):
            print(f"Estimator {i} parameters: {params}")
    else:
        # 単一のモデルの場合
        base_params = model.get_params()
        print("Single model parameters:", base_params)

    # ピリオドをアンダースコアに置き換えたファイル名
    cv_score_str = str(cv_score).replace(".", "_")

    # モデルの保存
    joblib.dump(model, f"/content/drive/MyDrive/yuto/project_directory/models/sub_model_{model_name}_{cv_score_str}.pkl")

    # jsonファイル作成
    data_to_save = {
        'columns': columns,
        f'{model_name}_params': base_params,
        'cv_score': cv_score,
        'lb_score': None
    }
    filename = f"/content/drive/MyDrive/yuto/project_directory/data/model_params/sub_model_{model_name}_{cv_score_str}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data_to_save, f, ensure_ascii=False, indent=4)

In [ ]:
ML_Record(df_train, model_lgbm, mae_score, now_str, model_name="lgbm_l1_cv")

All estimators have the same parameters.
Estimator 0 parameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.2661179203230455, 'importance_type': 'split', 'learning_rate': 0.04804216261746864, 'max_depth': 128, 'min_child_samples': 577, 'min_child_weight': 5.774925183375773, 'min_split_gain': 0.3914873364805788, 'n_estimators': 46022, 'n_jobs': None, 'num_leaves': 31, 'objective': 'l1', 'random_state': 42, 'reg_alpha': 0.0, 'reg_lambda': 0.0, 'subsample': 0.528846990441364, 'subsample_for_bin': 200000, 'subsample_freq': 0, 'metric': 'mae', 'colsample_bynode': 0.216985581379894, 'bagging_fraction': 0.9760006064193019, 'feature_fraction': 0.5336978769826026, 'max_bin': 1123, 'epsilon': 4.954717758504798, 'alpha': 0.5725841971738328, 'min_sum_hessian_in_leaf': 1.9378627484627984, 'boost_from_average': False, 'verbose': -1}
Estimator 1 parameters: {'boosting_type': 'gbdt', 'class_weight': None, 'colsample_bytree': 0.2661179203230455, 'importance_type': 'split', 'l